In [1]:
library(httr)
library(jsonlite)

## Prepare the interaction with EdelweissData

In [29]:
# Go to https://edelweissdata.com , log in and go to manage datasets and copy the access token with the button on the top right, then paste it here
access_token <- "" 

In [3]:
base_url <- "https://api.edelweissdata.com"

## Create a dataset in EdelweissData

In [4]:
request_payload <- list(name="Example dataset")

In [5]:
json_payload <- toJSON(request_payload, pretty=TRUE, auto_unbox=TRUE)
json_payload

{
  "name": "Example dataset"
} 

In [6]:
create_url <- paste(base_url, "datasets", "create", sep = "/")
create_url

[1] "https://api.edelweissdata.com/datasets/create"

In [7]:
res <- POST(create_url, body = json_payload, add_headers("Authorization" = paste("bearer", access_token)))
stop_for_status(res)
json <- content(res, "text")
create_reply <- fromJSON(json)
create_reply

$id
[1] "ae81c0a5-3368-41aa-9304-91df4ae83444"

$name
[1] "Example dataset"

$schema
NULL

$created
[1] "2020-09-07T10:45:14.9429781+00:00"

$description
[1] ""

$metadata
named list()

$dataSource
NULL

Now extract the DatasetID from the returned json

In [8]:
dataset_id <- create_reply$id
dataset_id

[1] "ae81c0a5-3368-41aa-9304-91df4ae83444"

## Upload the CSV data file

In [9]:
upload_url <- paste(base_url, "datasets", dataset_id, "in-progress", "data", "upload", sep = "/")
upload_url

[1] "https://api.edelweissdata.com/datasets/ae81c0a5-3368-41aa-9304-91df4ae83444/in-progress/data/upload"

In [12]:
res <- POST(upload_url,
            body = list(data = upload_file("small1.csv")),
            encode = "multipart",
            add_headers("Authorization" = paste("bearer", access_token))
           )
stop_for_status(res)
json <- content(res, "text")
upload_reply <- fromJSON(json)
upload_reply

$id
[1] "ae81c0a5-3368-41aa-9304-91df4ae83444"

$name
[1] "Example dataset"

$schema
NULL

$created
[1] "2020-09-07T10:45:14.9429780+00:00"

$description
[1] ""

$metadata
named list()

$dataSource
NULL

## Infer the schema

In [13]:
infer_url <- paste(base_url, "datasets", dataset_id, "in-progress", "schema", "infer", sep = "/")

In [14]:
res <- POST(infer_url,                        
            add_headers("Authorization" = paste("bearer", access_token))
           )
stop_for_status(res)

## Set a human readable description

In [15]:
description <- "Example data description. 
"

In [16]:
update_url <- paste(base_url, "datasets", dataset_id, "in-progress", sep = "/")

In [17]:
request_payload <- list(description=description)
json_payload <- toJSON(request_payload, pretty=TRUE, auto_unbox=TRUE)

In [18]:
res <- POST(update_url, body = json_payload, add_headers("Authorization" = paste("bearer", access_token)))
stop_for_status(res)

## Set the visibility

In [19]:
request_payload <- list(isPublic=TRUE)

In [20]:
json_payload <- toJSON(request_payload, pretty=TRUE, auto_unbox=TRUE)
json_payload

{
  "isPublic": true
} 

In [21]:
visibility_url <- paste(base_url, "datasets", dataset_id, "permissions", "visibility", sep = "/")
visibility_url

[1] "https://api.edelweissdata.com/datasets/ae81c0a5-3368-41aa-9304-91df4ae83444/permissions/visibility"

In [25]:
res <- POST(visibility_url, body = json_payload, add_headers("Authorization" = paste("bearer", access_token)))
stop_for_status(res)

## Publish the dataset

In [26]:
publish_url <- paste(base_url, "datasets", dataset_id, "in-progress", "publish", sep = "/")
request_payload <- list(changelog="Initial version")
json_payload <- toJSON(request_payload, pretty=TRUE, auto_unbox=TRUE)

In [27]:
res <- POST(publish_url, body = json_payload, add_headers("Authorization" = paste("bearer", access_token)))
stop_for_status(res)